## Preparación de datos para sistema recomendador proxy

En este *notebook* prepararemos los datos necesarios para evaluar un sistema recomendador externo con la librería `Elliot`. Se usarán los datasets de libros, usuarios y ratings.

### Creación de datasets de entrenamiento, validación y test

In [1]:
import os
import pandas as pd

# Ruta de datasets procesados
ready_path = os.path.join(os.getcwd(), "../..", "datasets", "ready")

# Lectura del dataset de ratings procesado
col_types = {
    'user_id': 'int32',
    'book_id': 'int32',
    'rating': 'float32',
}
ratings_df = pd.read_csv(ready_path + "/ratings.csv", dtype=col_types)
print("# de ratings:", ratings_df.shape[0])

# de ratings: 5515602


In [2]:
# Ruta de datasets en bruto
raw_path = os.path.join(os.getcwd(), "../..", "datasets", "raw")

# Lectura del dataset de libros en bruto (coincidirá con el procesado después de limpiarlo)
books_df = pd.DataFrame(pd.read_pickle(raw_path + "/books_raw.pkl"))
print("# de libros:", books_df.shape[0])

# de libros: 9467


Separamos el dataset de ratings en parte de entrenamiento y parte de test, en una proporción del 80% y 20%, respectivamente.

In [3]:
# Creamos los datasets de train (80%) y test (20%)
import sklearn.model_selection as model_selection

train, test = model_selection.train_test_split(ratings_df, test_size=0.2, random_state=42)
print("# de ratings en train (train + validation):", train.shape[0])
print("# de ratings en test:", test.shape[0])

# de ratings en train (train + validation): 4412481
# de ratings en test: 1103121


Además, en vista de modelos futuros, el dataset de entrenamiento será parte entrenamiento (70% del global) y parte validación (10% del global). Es fácil comprobar que se debe hacer un split local del dataset de entrenamiento en un 87.5% para entrenamiento *per se* y un 12.5% para validación, consiguiendo los porcentajes globales deseados.

In [4]:
# Creamos los datasets de train (87.5% -> 70% del global) y validación (12.5% -> 10% del global)
train_train, train_valid = model_selection.train_test_split(train, test_size=0.125, random_state=42)
print("# de ratings en train:", train_train.shape[0])
print("# de ratings en validation:", train_valid.shape[0])

# de ratings en train: 3860920
# de ratings en validation: 551561


### Entrenamiento del modelo (perfiles de usuario)

Para el modelo, crearemos un dataset de perfiles de usuario que codifique sus preferencias a partir del conjunto de datos de entrenamiento. Los perfiles de usuario estarán basados en los libros que haya puntuado. En concreto, se tendrán las mismas columnas que con los libros: `semantic_sbert`, `semantic_use` y `sentiment`. Pero, en este caso, cada una de las columnas corresponderá a una ponderación calculada de esta manera:

Tomaremos aquellos libros que haya valorado positivamente, es decir, con una puntuación mayor o igual que 0.75. Sea entonces $i_k$ el identificador del k-ésimo libro puntuado por el usuario de ese subconjunto de libros. El vector resultante es el normalizado de $$\frac{\sum_{k}^{}rating(i_k) * \vec{v}_{libro}(i_k)}{\sum_{k}^{}rating(i_k)}$$

In [5]:
# Creamos el dataframe de entrenamiento para el modelo a partir del dataset de train
train_df = pd.DataFrame(train, columns=['user_id', 'book_id', 'rating'])
train_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_df.head()

,user_id,book_id,rating
0,1,11,1.00
1,1,13,0.75
2,1,31,0.75
3,1,32,0.75
4,1,33,0.75


In [6]:
# Creamos un dataframe con los usuarios que aparecen en el dataset de train
users_df = pd.DataFrame(columns=['user_id'])
users_df['user_id'] = train_df['user_id'].unique()
users_df.sort_values(by=['user_id'], inplace=True)
users_df.reset_index(drop=True, inplace=True)
print("# de usuarios:", users_df.shape[0])

# de usuarios: 52371


In [7]:
# Obtenemos la información de los libros que han sido valorados por los usuarios del dataset de train
books_df = books_df[books_df['book_id'].isin(train_df['book_id'].unique())]
books_df.sort_values(by=['book_id'], inplace=True)
books_df.reset_index(drop=True, inplace=True)
print("# de libros:", books_df.shape[0])

# de libros: 9467


In [8]:
import numpy as np

def create_user_feature(user_id, feature, train_df: pd.DataFrame, books_df: pd.DataFrame):
    """
    Crea un vector normalizado de la característica dada a partir
    de los libros que al usuario le gustaron (rating >= 0.75). El vector
    es la suma ponderada de los vectores de la característica dada de
    dichos libros, donde los pesos son los ratings de los libros.

    ## Parámetros
    - user_id: ID del usuario.
    - param feature: Nombre de la característica (columna) a calcular.
    - train_df: `DataFrame` del dataset de entrenamiento.
    - books_df: `DataFrame` de libros en el dataset de entrenamiento.

    ## Retorna
    Vector normalizado de la característica dada.
    """
    # Ratings positivas del usuario
    user_ratings = train_df[train_df['user_id'] == user_id]
    user_likes = user_ratings[user_ratings['rating'] >= 0.75]
    # Libros que le gustaron al usuario
    user_books = books_df[books_df['book_id'].isin(user_likes['book_id'])]
    feature_array = np.array(user_books[feature])
    # Media ponderada de los vectores de la característica dada de los libros
    user_feature = np.sum([feature_array[i]*np.array(user_likes['rating'])[i] for i in range(len(feature_array))], axis=0)
    total_ratings_sum = np.sum(np.array(user_likes['rating']))
    if total_ratings_sum != 0:
        user_feature = user_feature/total_ratings_sum
    # Se retorna el vector normalizado
    norm = np.linalg.norm(user_feature)
    if norm == 0:
        return user_feature
    return user_feature/norm

In [9]:
# Ejemplo de creación de perfil de usuario (ID 1, característica 'sentiment')
create_user_feature(1, 'sentiment', train_df, books_df)

array([0.48328285, 0.02516897, 0.00446195, 0.01960389, 0.26584741,
       0.01181137, 0.0063063 , 0.01262651, 0.01212371, 0.06401394,
       0.01505276, 0.00546003, 0.00681106, 0.01059581, 0.00963669,
       0.00372715, 0.00863336, 0.01646627, 0.01775028, 0.00464807,
       0.80488975, 0.03050896, 0.00974535, 0.11301331, 0.00746177,
       0.01864082, 0.15745456, 0.04579422])

In [11]:
# Característica semantic_sbert de los usuarios
users_df['semantic_sbert'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'semantic_sbert', train_df, books_df))

In [12]:
# Característica semantic_use de los usuarios
users_df['semantic_use'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'semantic_use', train_df, books_df))

In [13]:
# Característica sentiment de los usuarios
users_df['sentiment'] = users_df['user_id'].apply(lambda x: create_user_feature(x, 'sentiment', train_df, books_df))

Para posibles usos recurrentes, guardamos nuestro modelo KNN, ya entrenado, en almacenamiento local. Realmente se estará guardando un dataset de perfiles de usuario obtenido a partir del 80% del dataset de ratings correspondiente al entrenamiento.

In [22]:
# Ruta de almacenamiento de modelos
models_path = os.path.join(os.getcwd(), "../..", "models")

# Guardamos el modelo KNN entrenado (dataset de usuarios)
users_df.to_pickle(models_path + "/knn_users.pkl")

### Predicciones a partir del modelo

Para hacer las predicciones, necesitamos crear una función de similitud que el modelo utilizará para encontrar los k libros más similares en función del perfil de usuario. Como tanto los libros como los perfiles de usuario cuentan con los mismos atributos (`semantic_sbert`, `semantic_use` y `sentiment`) la función de similitud podrá utilizarse para comparar libros con libros, usuarios con usuarios o libros con usuarios (y viceversa).

La fórmula propuesta es la siguiente: $$S(x, y) = w * S_C(x_{sem}, y_{sem}) + (1 - w) * S_C(x_{sent}, y_{sent}),$$ donde $w$ es un peso para la ponderación entre la parte semántica y la parte de sentiment analysis y $S_C$ es la similitud coseno entre los vectores.

In [14]:
# Definimos la función de similitud entre dos objetos
def sem_sent_sim(item1, item2, sem_option='semantic_sbert', sem_w=0.75) -> float:
    """
    Calcula la similitud entre un objeto y otro.
    Los objetos deben tener un vector `'semantic'` y otro `'sentiment'`, 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    sim(item1, item2) = sem_w * cos_sim(item1[sem_option], item2[sem_option]) + 
    (1 - sem_w) * cos_sim(item1['sentiment'], item2['sentiment'])

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    La similitud entre ambos objetos.
    """
    # Obtenemos el campo semántico de cada objeto
    sem1 = item1[sem_option]
    sem2 = item2[sem_option]
    # Obtenemos el campo de sentimiento de cada objeto
    sent1 = item1['sentiment']
    sent2 = item2['sentiment']
    # Calculamos la similitud
    return sem_w * np.dot(sem1, sem2) + (1 - sem_w) * np.dot(sent1, sent2)

def sem_sent_dist(item1, item2, sem_option='semantic_sbert', sem_w=0.75) -> float:
    """
    Calcula la distancia entre un objeto y otro.

    Los objetos deben tener un vector 'semantic' y otro 'sentiment', 
    previamente normalizados.

    La similitud sigue la siguiente fórmula:

    dist(item1, item2) = 1 - sim(item1, item2)

    ## Parámetros:
    - item1: Primer objeto.
    - item2: Segundo objeto.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    La distancia entre ambos objetos.
    """
    return 1 - sem_sent_sim(item1, item2, sem_option, sem_w)

In [15]:
def k_nearest(item, train_df: pd.DataFrame, books_df: pd.DataFrame, k=10, sem_option='semantic_sbert', sem_w=0.75):
    """
    Calcula los k vecinos más cercanos a un objeto.

    ## Parámetros:
    - item: Objeto del que queremos obtener los k vecinos más cercanos.
    - train_df: `DataFrame` del dataset de entrenamiento.
    - books_df: `DataFrame` de libros en el dataset de entrenamiento.
    - k: Número de vecinos más cercanos que queremos obtener.
    - sem_option: Opción de contenido semántico. Por defecto su valor
    es `'semantic_sbert'` (modelo SBERT), pero también puede valer `'semantic_use'`
    (modelo USE).
    - sem_w: Peso del contenido semántico. Por defecto su valor es 0.75.

    ## Retorna:
    k tuplas (book_id, similitud) con los k vecinos más cercanos al objeto.
    """
    if 'user_id' in item:
        # Ratings positivas del usuario
        user_ratings = train_df[train_df['user_id'] == item['user_id']]
        user_likes = user_ratings[user_ratings['rating'] >= 0.75]
        # Libros que le gustaron al usuario
        user_books = books_df[books_df['book_id'].isin(user_likes['book_id'])]
        # Eliminamos los libros que ya ha valorado el usuario positivamente
        books_df = books_df[~books_df['book_id'].isin(user_books['book_id'])]
    # Calculamos las distancias entre el objeto y todos los demás
    sim_items = books_df.apply(lambda x: sem_sent_sim(item, x, sem_option, sem_w), axis=1)
    # Obtenemos los k vecinos más cercanos
    nearest = sim_items.sort_values(ascending=False).head(k)
    return list(nearest.items())

In [16]:
# Ejemplo de obtención de los 10 libros más cercanos a un usuario (ID 1)
k_nearest(users_df[users_df['user_id'] == 1].iloc[0], train_df, books_df)

[(2983, 0.8524908574044785),
 (3093, 0.8158039702323813),
 (5456, 0.8124956792103959),
 (257, 0.810777558399494),
 (639, 0.8085742151490068),
 (4535, 0.8075308592478379),
 (5460, 0.8073959101906684),
 (1126, 0.8061645801230144),
 (6995, 0.8024756249884195),
 (2547, 0.8004604821814822)]

In [18]:
# Calculamos los k libros más similares (de todo el dataset de libros) a cada perfil de usuario (sin contar los ya positivamente valorados)
k = 50
predictions_df = users_df.copy()
predictions_df = predictions_df.drop(columns=['semantic_sbert', 'semantic_use', 'sentiment'])
predictions_df['nearest'] = predictions_df['user_id'].apply(lambda x: k_nearest(users_df[users_df['user_id'] == x].iloc[0], train_df, books_df, k=k))

In [19]:
predictions_df.head()

,user_id,nearest
0,1,"[(2983, 0.8524908574044785), (3093, 0.81580397..."
1,2,"[(7013, 0.7838041901097742), (5153, 0.76307008..."
2,4,"[(2983, 0.8305538651482938), (7013, 0.82558547..."
3,5,"[(7189, 0.8475733224361469), (3907, 0.84396841..."
4,6,"[(2983, 0.8135763730876129), (5153, 0.80119439..."


In [20]:
# Transformamos la columna 'nearest' en dos columnas: 'book_id' y 'prediction'
predictions_df = predictions_df.explode('nearest')
predictions_df[['book_id', 'prediction']] = pd.DataFrame(predictions_df['nearest'].tolist(), index=predictions_df.index)
predictions_df = predictions_df.drop('nearest', axis=1)

In [21]:
predictions_df.head()

,user_id,book_id,prediction
0,1,2983,0.852491
0,1,3093,0.815804
0,1,5456,0.812496
0,1,257,0.810778
0,1,639,0.808574


### Creación de los ficheros para Elliot

Una vez calculadas todas las predicciones de los libros por usuario a partir del modelo, guardaremos todos los datasets (entrenamiento, validación y test) así como las propias predicciones en formato .tsv, de tal forma que puedan ser utilizados en `Elliot`.

In [23]:
# Creamos el dataframe del dataset de test (para darle un formato ordenado)
test_df = pd.DataFrame(test, columns=['user_id', 'book_id', 'rating'])
test_df.sort_values(by=['user_id', 'book_id'], inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [24]:
# Creamos el dataframe de entrenamiento 70% (para darle un formato ordenado)
train_train_df = pd.DataFrame(train_train, columns=['user_id', 'book_id', 'rating'])
train_train_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_train_df.reset_index(drop=True, inplace=True)

In [25]:
# Creamos el dataframe de validación 10% (para darle un formato ordenado)
train_valid_df = pd.DataFrame(train_valid, columns=['user_id', 'book_id', 'rating'])
train_valid_df.sort_values(by=['user_id', 'book_id'], inplace=True)
train_valid_df.reset_index(drop=True, inplace=True)

In [26]:
# Ruta de datasets de entrenamiento
training_path = os.path.join(os.getcwd(), "../..", "datasets", "training")

# Guardamos el dataset de predicciones así como los de entrenamiento, test y validación
predictions_df.to_csv(training_path + "/predictions.tsv", sep='\t', header=None, index=False)
train_df.to_csv(training_path + "/train.tsv", sep='\t', header=None, index=False)
test_df.to_csv(training_path + "/test.tsv", sep='\t', header=None, index=False)
train_train_df.to_csv(training_path + "/train_train.tsv", sep='\t', header=None, index=False)
train_valid_df.to_csv(training_path + "/train_valid.tsv", sep='\t', header=None, index=False)